In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import util
import os
import engutil

def loudspeaker_ode(t, x, u_func, params, polys):
    i_curr = x[0]  
    i_creep = x[1] 
    disp = x[2]    
    vel = x[3]    
    
    voltage = u_func(t) # input voltage
    
    # values from Bl, K and Le polynomials
    val_Bl = polys['Bl'](disp)
    val_K  = polys['K'](disp)
    val_Le_base = polys['Le'](disp) * polys['Li'](disp)
    val_Le = max(val_Le_base, 1e-9) # ensure there will be no divide by 0
    val_Le_deriv = val_Le_base.deriv()
    Le_nominal = polys['Le'](0) # x = 0 value of Le
    ratio = val_Le / Le_nominal
    
    # scale R2 and L2 accordingly
    val_R2 = params['R20'] * ratio
    val_L2 = params['L20'] * ratio
    
    R_e = params["R_e"]
    R_m = params["R_m"]


    di_dt = (val_R2*i_creep + voltage - i_curr*(val_R2 + R_e) - vel*(val_Bl + val_Le_deriv*i_curr))/L_e_star
    di2_dt = (-L_e_star*i_creep*(val_R2 + dL_2/dd*vel) + dL_2/di*i_creep*(-voltage + vel*(Bl + dL_e/dd*i_curr)) + i_curr*(L_e_star*val_R2 + R_e*dL_2/di*i_crep))/(val_L2*L_e_star)
    ddisp_dt = vel
    dvdt = (-K_m*disp - R_m*vel + dL_2/dd*i_creep**2/2 + i_curr*(2*Bl + dL_e/dd*i)/2)/M_m


    #     di_dt = (val_R2*i_creep + voltage - i_curr*(val_R2 + R_e) - vel*(val_Bl + dL_e/dd*i_curr))/L_e_star
    # di2_dt = (-L_e_star*i_creep*(val_R2 + dL_2/dd*vel) + dL_2/di*i_creep*(-voltage + vel*(Bl + dL_e/dd*i_curr)) + i_curr*(L_e_star*val_R2 + R_e*dL_2/di*i_crep))/(val_L2*L_e_star)
    # ddisp_dt = vel
    # dvdt = (-K_m*disp - R_m*vel + dL_2/dd*i_creep**2/2 + i_curr*(2*Bl + dL_e/dd*i)/2)/M_m






    # equation 0: di/dt = (1/Le) * (V - (Re+R2)i + R2*i2 - Bl*v)
    di_dt = (1.0 / val_Le) * (voltage - (params['Re'] + val_R2)*i_curr + val_R2*i_creep - val_Bl*vel)
    
    # equation 1: di2/dt = (1/L2) * (R2*i - R2*i2)
    di2_dt = (val_R2 / val_L2) * (i_curr - i_creep)
    
    # equation 2: d(disp)/dt = vel
    ddisp_dt = vel
    
    # equation 3: d(vel)/dt = (1/Mm) * (Bl*i - K*x - Rm*v)
    dvel_dt = (1.0 / params['Mm']) * (val_Bl*i_curr - val_K*disp - params['Rm']*vel)
    
    return [di_dt, di2_dt, ddisp_dt, dvel_dt]

# setup params
Bl = 6.837193705842441
R_e = 4.822146670305587
R_m = 2.2833222074164836
K_m = 1107.8705431240523
L_e0 = 0.000305207582223968
M_m = 0.020004906958875274
L_20 = 0.0004325286577462285
R_20 = 2.504536634680621

Bln = [-2615563718.9455967, -3023090.98130258, -9835.616494456142, 21.171849188494026, Bl]
Kn  = [1724760519892.831, -5658865945.33859, 35715360.4027017, -210546.86882586573, K_m]
Ln  = [20081.188591764963, -319.17779393260366, 0.8155679039802962, -0.01018945289267653, L_e0]
Li  = [-0.0012237427320541485, -0.0014189369785025077, 0.007354393550658496, 0.005954177167801685, 1]

# polys = {
#     'Bl': np.poly1d(Bln),
#     'K':  np.poly1d(Kn),
#     'Le': np.poly1d(Ln),
#     'Li': np.poly1d(Li)
# }

params = {
    'Re': R_e, 'Rm': R_m, 'Mm': M_m, 'R20': R_20, 'L20': L_20, 'Le_nom': L_e0
}

# for calculating min fs
F_linear_static = np.array([
    [-(R_e + R_20)/L_e0,  R_20/L_e0,    0.0,         -Bl/L_e0],
    [R_20/L_20,          -R_20/L_20,    0.0,          0.0 ],
    [0.0,                   0.0,            0.0,          1.0],
    [Bl/M_m,                0.0,           -K_m/M_m,   -R_m/M_m]
])

minimum_fs = util.calculate_min_fs(F_linear_static)
print(f"Minimum Linear fs: {minimum_fs:.2f}")

# duration 2 and 3 * min fs tog ~4min
duration = 5.0
fs = int(5 * minimum_fs)
t_eval = np.linspace(0, duration, int(fs*duration))
u = util.generate_pink_noise(len(t_eval), fs, fmin=1)

# u(t) function - ie to be able to find values between sample values.. 
u_func = interp1d(t_eval, u, kind='linear', fill_value="extrapolate")


scenarios = [
    {
        "name": "01_nonlinear_full",
        "polys": {
            'Bl': np.poly1d(Bln),
            'K':  np.poly1d(Kn),
            'Le': np.poly1d(Ln),
            'Li': np.poly1d(Li)
        }
    }
]

output_dir = "data/part2d_simulations"
os.makedirs(output_dir, exist_ok=True)

x0 = [0, 0, 0, 0] # Initial conditions

for case in scenarios:
    print(f"Running simulation: {case['name']} ...")
    
    current_polys = case['polys']
    
    sol = solve_ivp(
        fun=loudspeaker_ode,
        t_span=(0, duration),
        y0=x0,
        t_eval=t_eval,      
        args=(u_func, params, current_polys), 
        method='RK45',      
        rtol=1e-3,       # 1e-3   (was 5)
        atol=1e-6        # 1e-6 ( was 8)
    )
    
    # Organize data for saving
    # Structure: [Time, Current, Creep_Current, Displacement, Velocity]
    # Transpose (.T) ensures rows represent time steps
    data_to_save = np.vstack((sol.t, u, sol.y)).T
    
    filename = f"{output_dir}/{case['name']}.csv"
    
    # Save with header
    np.savetxt(
        filename, 
        data_to_save, 
        delimiter=",", 
        header="time,i,i_creep,disp,vel",
        comments="" # Removes the '#' usually added to headers by numpy
    )
    
    print(f"Saved: {filename}")

print("All simulations complete.")



